In [1]:
import re
import reprlib

In [5]:
class Sentence:
    RE_WORD = re.compile('\w+')
    def __init__(self, text):
        self.text = text
        self.words = self.RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

In [6]:
s = Sentence("The time has come, the Walrus said,")

In [7]:
for w in s:
    print(w)

The
time
has
come
the
Walrus
said


In [8]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

**序列可迭代的原因：iter函数**

解释器需要迭代对象x时，会自动调用iter(x)，任何Python序列都可以迭代的原因是，他们都实现了`__getitem__()`方法。

内置的`iter()`函数按照一下步骤发挥作用：

1. 检查对象是否实现了`__iter__()`方法，如果实现了，就调用它，获取一个迭代器
2. 如果没有实现`__iter__()`方法，但是实现了`__getitem__()`方法，Python会创建一个迭代器，尝试按顺序获取元素
3. 如果以上尝试都宣告失败，Python抛出TypeError异常

检查对象是否可迭代，最准确的方法是使用`iter(x)`函数，如果不可迭代，再处理TypeError异常。最好的方式是调用`isinstance(x, abc.Iterator)`

### 可迭代对象

使用 iter 内置函数可以获取迭代器的对象。如果对象实现了能返回迭代器的 `__iter__` 方法，那么对象就是可迭代的。序列都可以迭代；实现了 `__getitem__` 方法，而且其参数是从零开始的索引，这种对象也可以迭代。

可迭代的对象和迭代器之间的关系：Python 从可迭代的对象中获取迭代器。 

In [9]:
s = 'ABC' 
for char in s:  
    print(char) 

A
B
C


字符串'ABC'是可迭代对象。

In [ ]:
class Sentence:
    RE_WORD = re.compile("\w+")
    
    def __init__(self, text):
        self.text = text
        self.wprds = RE_WORD.findall(text)
        
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(se;f.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)
    

class SentenceIterator:
    
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.word[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        
    def __iter__(self):
        return self

迭代器应该实现 `__next__` 和 `__iter__` 两个方法，这么做能让迭代器通过issubclass(yourIteratorObject, abc.Iterator)测试。

实现一个**可迭代对象**，需要实现`__iter__()`方法，该方法返回一个新的迭代器。

而实现一个**迭代器**，要实现`__iter__()`方法和`__next__()`方法，其中`__next__()`方法返回单个元素，`__iter__()`犯法返回迭代器本身。

**迭代器可以迭代，但是可迭代对象不是迭代器。可迭代的对象一定不能同时是自身的迭代器，也就是说，可迭代的对象必须实现`__iter__()`方法，但是不能实现`__next__()`方法。另一方面，迭代器应该一致可以迭代，迭代器的`__iter__()`方法应该返回本身。**

## 使用生成器函数代替SentenceIterator类

In [10]:
class Sentence:
    
    RE_WORD = re.compile('\w+')
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        # 不管有没有 return 语句，生成器函数都不会抛出 StopIteration 异常，而是在生成完全部值之后会直接退出
        return

只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函数。调用生成器函数时，会返回一个生成器对象。也就是说，生成器函数是生成器工厂。

生成器函数会创建一个生成器对象，包装生成器函数的定义体。把生成 器传给 next(...) 函数时，生成器函数会向前，执行函数定义体中的 下一个 yield 语句，返回产出的值，并在函数定义体的当前位置暂停。

## 惰性实现

In [11]:
class Sentence:
    RE_WORD = re.compile('\w+')
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)
    
    # 将__iter__()定义为一个生成器函数
    def __iter__(self):
        # 惰性匹配，不一次性构建self.words列表
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 用生成器表达式代替yield

**生成器是迭代器。**

In [13]:
class Sentence:
    RE_WORD = re.compile('\w+')
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)
    
    # 使用生成器表达式代替生成器函数
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## 使用生成器函数实现特殊的`__iter__()`方法

In [17]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self):
        # type(self.begin + self.step)是为了得到self.begin或者self.end其中某一个的数据类型，并将其作为返回结果的类型
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        self.index = 0
        while forever or result < self.end:
            yield result
            self.index += 1
            result = self.begin + self.index * self.step

In [18]:
ap = ArithmeticProgression(0, 1, 3)

In [19]:
list(ap)

[0, 1, 2]

## 多利用内置的itertools模块

itertools.takewhile 会生成一个生成器并返回，返回的生成器在指定的条件计算结果为 False时停止。

In [20]:
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

In [21]:
import numpy as np

In [22]:
a = np.arange(10)
b = np.arange(10)
np.vstack([a,b])

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

## 内置iter()函数

iter 函数还有一个鲜为人知的用法：传入两个参数，使用常规的函数或任何可调用的对象创建迭代器。这样使用时，第一个参数必须 是可调用的对象，用于不断调用（没有参数），产出各个值；第二个值是哨符，这是个标记值，当可调用的对象返回这个值时，触发迭代器抛出 StopIteration 异常，而不产出哨符

In [3]:
import random
def sample():
    return random.randint(1, 6)

# 掷6个面的色子，直到得到数字1为止
sampler_1 = iter(sample, 1)

for num in sampler_1:
    print(num)

2
6
3
2
5
6


有个实用的例子。这段代码逐行读取文件，直到遇到空行或者到达文件末尾为止：

In [ ]:
with open('data.txt') as fp:
    for line in iter(fp.readline, '\n'):
        process_func(line)